<a href="https://colab.research.google.com/github/dowrave/Tensorflow_Basic/blob/main/220516_Graph_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 텐서플로우가 님 코드를 그래프로 바꿔주게 할 거임
- 그래프가 저장, 표현, 모델 가속화에 어떻게 쓰이는지 알랴드림
- `tf.fuction`의 큰 Overview다~

# 그래프
- 이전 가이드에서 텐서플로우는 즉시 실행되었음.
- 그래프는 파이썬 외부에서 이식성을 가능하게 하며 텐서 계산을 `tf.Graph` 혹은 그래프라고 하는 Tensorflow 그래프로 실행됨을 의미함
- `tf.Operation` : 계산의 단위 객체
- `tf.Tensor` : 연산 간 흐르는 데이터의 단위
- `tf.Graph` : 데이터 구조
  - 그래프는 데이터 구조이므로 파이썬 코드 없이 저장, 실행, 복원할 수 있다.

## 그래프의 이점
- 유연성 향상 : 모바일 앱, 임베디드, 백엔드 서버와 같은, 파이썬 인터프리터가 없는 환경에서 tf 그래프를 사용할 수 있다. 
- 다음 변환들을 수행할 수 있다.
  - 계산에서 상수 노드들을 접어 텐서 값을 정적으로 추론
  - 독립 계산의 하위 부분을 분리, 스레드 or 기기 간 분할
  - 공통 하위 표현식을 제거, 산술 연산을 단순화함
- 전체 최적화 시스템으로 `Grappler`가 있다.

- 즉 그래프는 tf가 <b> 빠르게, 병렬로, 효율적으로 여러 기기에서 실행</b>할 때 유용하다.

## 그래프 이용하기
- `tf.function`을 직접 호출 or 데코레이터로 사용
- `tf.function`은 일반 함수를 입력으로 받아 `Function`을 반환한다. 이는 파이썬 함수로부터 tf 그래프를 빌드하는 Python Callable이다.

In [ ]:
import tensorflow as tf
import timeit
from datetime import datetime

def a_regular_function(x, y, b):
  x = tf.matmul(x, y)
  x = x + b
  return x

# 위 함수를 tf.function으로 묶어서 그래프로 지정함.
a_function_that_uses_a_graph = tf.function(a_regular_function)

x1 = tf.constant([[1., 2.]])
y1 = tf.constant([[2.], [3.]])
b1 = tf.constant(4.)

orig_value = a_regular_function(x1, y1, b1).numpy()
# graph는 파이썬 함수처럼 호출할 수 있다.
tf_function_value = a_function_that_uses_a_graph(x1, y1, b1).numpy()
assert(orig_value == tf_function_value)

- `Function`은 뭐 없어 보이지만 하나의 API 뒤에서 여러 `tf.Graph`를 캡슐화한다.이런 이유로 `Function`은 속도 & 배포 가능성과 같은, 그래프 실행의 이점을 제공한다.

In [ ]:
def inner_function(x, y, b):
  x = tf.matmul(x, y)
  x = x + b
  return x

@tf.function
def outer_function(x): # 얘는 tf.Graph를 캡슐화하게 됨
  y = tf.constant([[2.], [3.]])
  b = tf.constant(4.)

  return inner_function(x, y, b)

outer_function(tf.constant([[1., 2.]])).numpy()

## Python 함수의 그래프 변환
- Tensorflow 함수와 달리 Python 함수는 그래프의 일부가 되기 위해 추가적인 단계를 거쳐야 한다. 이는 `tf.autograph`라는 라이브러리를 사용하여 Python 코드를 그래프 생성 코드로 변환한다.

In [ ]:
def simple_relu(x):
  if tf.greater(x, 0):
    return x
  else:
    return 0

tf_simple_relu = tf.function(simple_relu)

print("First branch with graph : ", tf_simple_relu(tf.constant(1)).numpy())
print("Second branch with graph : ", tf_simple_relu(tf.constant(-1)).numpy())

In [ ]:
# autograph 사용
print(tf.autograph.to_code(simple_relu))

In [ ]:
# 이건 그냥 그래프 자체
print(tf_simple_relu.get_concrete_function(tf.constant(1)).graph.as_graph_def())

### 다형성
- 하나의 Function으로 다수의 그래프 만들기
- `tf.Graph`는 특정 유형의 입력(특정 `dtype`의 텐서, 동일한 `id()`를 가진 객체)에 특화되어 있다.
- `tf.Graph` 입력의 `dtypes`와 `shape`는 입력 서명 : 서명 이라고 한다.
- `Function`은 `tf.Graph`를 `ConcreteFunction`에 저장한다. `ConcreteFunction`은 `tf.Graph`를 감싸는 래퍼다.

In [ ]:
@tf.function
def my_relu(x):
  return tf.maximum(0., x)

# 이 3개는 각각 새로운 tf.Graph를 생성함 (2, 3은 dtype이 다르기 때문!)
print(my_relu(tf.constant(5.5)))
print(my_relu([1, -1]))
print(my_relu(tf.constant([3., -3.])))

In [ ]:
# 이미 동일한 value, dtype이 있는 그래프 객체가 위에서 생성되었기 때문에 얘네는 그래프가 새로 생성되지 않음
print(my_relu(tf.constant(-2.5))) # 서명이 tf.constant(5.5)와 동일함 : dtype, shape
print(my_relu(tf.constant([-1., 1.]))) # 서명이 tf.constant([3. -3.])과 동일함

### 다형성
- 여러 그래프로 뒷받침되기 떄문에 `Function`은 다형성이다. 그 결과, 단일 `tf.Graph`로 나타낼 수 있는 것보다 더 많은 입력 유형을 지원하고, `tf.Graph`가 더 우수한 성능을 갖도록 최적화할 수 있다.

In [ ]:
# 3개의 ConcreteFunction이 1개의 my_relu에 있다. Concretefunction은 return type, shape를 안다
print(my_relu.pretty_printed_concrete_signatures())

## tf.function 사용하기
- 위에선 파이썬을 `tf.function`을 이용해 그래프로 만들었다.
- 실제로는 `tf.function`은 Tricky하게 쓸 수 있다.

### 그래프 실행 vs 즉시 실행

In [ ]:
@tf.function
def get_MSE(y_true, y_pred):
  sq_diff = tf.pow(y_true - y_pred, 2) # element wise, a**b
  return tf.reduce_mean(sq_diff)


In [ ]:
y_true = tf.random.uniform([5], maxval = 10, dtype = tf.int32)
y_pred = tf.random.uniform([5], maxval = 10, dtype = tf.int32)
print(y_true, y_pred)

In [ ]:
# 기본 : tf.function은 그래프로 실행됨
get_MSE(y_true, y_pred)

In [ ]:
# Python 함수로 실행되는지 체크 - 이는 Function의 역할을 해제시킬 때 스위치처럼 이용되는 코드임
tf.config.run_functions_eagerly(True)
get_MSE(y_true, y_pred)

In [ ]:
# 잘 작동하는 게 확인되었다면 다시 되돌려줄 것
tf.config.run_functions_eagerly(False)

### 그래프와 함수에서 다르게 작동하는 경우

In [ ]:
@tf.function
def get_MSE(y_true, y_pred):
  print("Calculating MSE!")
  sq_diff = tf.pow(y_true - y_pred, 2)
  return tf.reduce_mean(sq_diff)

In [ ]:
# 3회 호출 = print도 3회 호출 but 1회만 호출되었음
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)

- 설들갑 : `print`는 `Function`이 원래 코드를 실행할 때 실행된다. `트레이싱`을 통해 그래프가 생성되는데, 이는 텐서플로우 연산을 그래프로 캡처하고 `print`는 캡처하지 않는다.

In [ ]:
# 비교 : config.run_functions_eagerly를 꺼보자
tf.config.run_functions_eagerly(True)

error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)

tf.config.run_functions_eagerly(False)

- Function에 대한 다른 차이점들이 있으며 이는 뒤에서 `tf.function`탭에서 또 다룰 거임
- print의 경우 둘 다 3번 출력시키고 싶다면 `tf.print`를 이용하라고 함.
- print는 파이썬의 부작용이라는 언급 또한 있다.

### tf.function을 사용하는 몇 가지 팁
- `tf.config.run_functions_eagerly()`를 통해 즉시 실행 - 그래프 실행 사이를 자주 전환하여 두 모드가 언제 어떻게 달라지는지 정확히 파악하기
- 파이썬 함수 외부에서 `tf.Variable`을 실행하고 내부에서 수정하기. `keras.layers`, `keras.Model`, `keras.optimizers` 등 `tf.Variable`을 사용하는 객체 모두 마찬가지다.
- `tf.Variables` 나 케라스 객체를 제외하고 외부 파이썬 변수에 종속되는 함수 작성을 피한다.
- 입력을 텐서 or 기타 텐서플로우 유형을 사용하는 함수를 작성하는 것이 좋다. 다른 객체 유형을 전달할 수 있으나, 주의해야 한다.
- 성능 이점을 극대화하기 위해 `tf.function`하에서 계산이 가능한 많이 포함되도록 한다. 전체 훈련 스텝 or 루프를 되풀이한다든가.

## 속도 향상 확인하기
- 일반적으로 `tf.function`은 코드의 성능을 향상시키나, 정도는 케바케임.
- 작은 계산은 그래프를 호출하는 오버헤드에 의해 지배될 수 있다.

In [ ]:
x = tf.random.uniform(shape = [10, 10], minval = -1, maxval = 2, dtype = tf.dtypes.int32)

def power(x, y):
  result = tf.eye(10, dtype = tf.dtypes.int32)
  for _ in range(y):
    result = tf.matmul(x, result)

  return result

print("Eager Execution : ", timeit.timeit(lambda : power(x, 100), number = 1000))

In [ ]:
# function 씌워서 보기
power_as_graph = tf.function(power)
print("Eager Execution : ", timeit.timeit(lambda : power_as_graph(x, 100), number = 1000))

- `tf.function`은 주로 훈련 루프에서 속도를 올리는 데 많이 사용된다.
- 과중하고 작은 텐서를 많이 쓴다면 `tf.function(jit_compile = True)`를 시도해볼 수 있다.

#### 성능과의 상충 관계
- 그래프를 실행하는 시간보다 만드는 시간이 오래 걸리기도 한다.
- 트레이싱으로 인해 모델의 성능이 저하되는 케이스는 피해야 함
- 모델 크기에 관계 없이 빈번한 트레이싱은 피해야 한다. `tf.function` 가이드에 리트레이싱을 피하기 위한 입력 사양 설정 & 텐서 인수 사용 방법이 있음

#### Function이 트레이싱 수행하는 경우 파악하기
- 코드에 `print`문 추가
- 인자(Argument)로 <b>tf 객체를 전달하는 경우 리트레이싱이 발생하지 않음</b>
- <b>한편 파이썬 객체를 전달한 경우 리트레이싱이 발생</b>함

In [ ]:
@tf.function
def a_function_with_python_side_effect(x):
  print("Tracing!") # Tracing 마다 실행됨
  return x * x + tf.constant(2)

# trace 1
print(a_function_with_python_side_effect(tf.constant(2)))
# trace X
print(a_function_with_python_side_effect(tf.constant(3)))

In [ ]:
print(a_function_with_python_side_effect(2))
print(a_function_with_python_side_effect(3))